In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas.errors import InvalidIndexError
import os
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import Lambda, Dense
import keras.backend as K
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm, datasets
import numpy as np
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from pandas import DataFrame
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import tensorflow_hub as hub
from pandas import DataFrame
from pandas.errors import InvalidIndexError
import os
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
import seaborn as sns
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def stemming_and_lemmatization(sentence):
    stemmer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    newSentence = re.sub(r'\W', ' ', sentence)
    newSentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', newSentence)
    newSentence = re.sub(r'\^[a-zA-Z]\s+', ' ', newSentence)
    newSentence = re.sub(r'\s+', ' ', newSentence, flags=re.I)
    newSentence = newSentence.lower()
    newSentence = newSentence.split()
    newSentence = [w for w in newSentence if not w in stop_words]
    newSentence = [stemmer.lemmatize(word) for word in newSentence]
    newSentence = ' '.join(newSentence)
    return newSentence

def isNaN(string):
    return string != string

def create_vectors_dataframe(df):
    model = KeyedVectors.load_word2vec_format('/home/kapil/SJSU-Acad/MastersProject/codebase/data/GoogleNews-vectors-negative300.bin', binary=True)
    for index, row in df.iterrows():
        if not isNaN(df['title'][index]):
            df['title'] = model[df['title']]
        if not isNaN(df['abstractText'][index]):
            df['abstractText'] = model[df['abstractText']]
        if not isNaN(df[' Introduction'][index]):
            df[' Introduction'] = model[df[' Introduction']]
        if not isNaN(df[' Conclusion'][index]):
            df[' Conclusion'] = model[df[' Conclusion']]
    return df

def preprocess_reviews_dataframe(df):
    for index, row in df.iterrows():
        print(index)
        row['abstractText'] = stemming_and_lemmatization(str(row['abstractText']))
        row['title'] = stemming_and_lemmatization(str(row['title']))
    return df

def preprocess_dataframe(df):
    for index, row in df.iterrows():
        print(index)
        row['abstractText'] = stemming_and_lemmatization(str(row['abstractText']))
        row['title'] = stemming_and_lemmatization(str(row['title']))
        row[' Introduction'] = stemming_and_lemmatization(str(row[' Introduction']))
        row[' Related Work'] = stemming_and_lemmatization(str(row[' Related Work']))
        row[' Conclusion'] = stemming_and_lemmatization(str(row[' Conclusion']))
        row[' Experiments'] = stemming_and_lemmatization(str(row[' Experiments']))
        row[' Results'] = stemming_and_lemmatization(str(row[' Results']))
        row[' Discussion'] = stemming_and_lemmatization(str(row[' Discussion']))
    return df

def tokenize_dataframe(df):
    for index, row in df.iterrows():
        df['abstractText'][index] = word_tokenize(str(df['abstractText'][index]))
        df['title'][index] = word_tokenize(df['title'][index])
        df[' Introduction'][index] = word_tokenize(df[' Introduction'][index])
        df[' Conclusion'][index] = word_tokenize(df[' Conclusion'][index])
        df[' Experiments'][index] = word_tokenize(df[' Experiments'][index])
        df[' Related Work'][index] = word_tokenize(df[' Related Work'][index])
        df[' Discussion'][index] = word_tokenize(df[' Discussion'][index])
        df[' Results'][index] = word_tokenize(df[' Results'][index])

    return df

def doc_to_vec(df):
    for column in df:
        if column == 'abstractText' or column == 'title' or column == ' Introduction' or column == ' Related Work' or column == ' Experiments' or column == ' Results' or column == ' Conclusion' or column == ' Discussion':
            print(df[column])
            docs = [TaggedDocument(str(df[column][i]).split(' '), [i])
                         for i, doc in enumerate(df[column])]
            model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs=40)
            model.build_vocab(docs)
            model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)

            vec = [model.infer_vector((df[column][i].split(' ')))
                        for i in range(0, len(df[column]))]

            df[column] = np.array(vec).tolist()
    return df

def hub_to_vec(df):
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
    embeddings = embed(df['abstractText'])
    use = np.array(embeddings).tolist()
    df['use'] = use
    return df


In [ ]:
def read_json(path_to_json):
    df = pd.read_json(path_to_json)
    name = df.iloc[0]['name']
    df = df.T
    df = df.iloc[1:]
    df.insert(0, 'name', name)
    newdf = DataFrame(df.iloc[0]['sections'])
    newdf = newdf.T
    newdf.iloc[0] = newdf.iloc[0].str.replace('\d+|\.', '')
    newdf.columns = newdf.iloc[0]
    newdf = newdf.iloc[1:]
    newdf.insert(0, 'name', name)
    result = pd.merge(df, newdf, on=["name"])
    referenceMentions_count = str(list(result["referenceMentions"])[0]).lower().split().count('\'context\':')
    references_count = str(list(result["references"])[0]).lower().split().count('\'year\':')
    result['references'] = references_count
    result['referenceMentions'] = referenceMentions_count
    return result

def create_dataframe_from_parsed_pdfs():
    rootdir = './drive/MyDrive/data_to_drive/'
    columns_list = ["name", "abstractText", "authors", "creator", "emails", "referenceMentions", "references", "title",
                    "year", " Introduction", " Related Work", " Experiments", " Results", " Conclusion", " Discussion", "accepted"]
    df = pd.DataFrame(columns=columns_list)
    count = 0
    invalidIndexErrorCount = 0
    indexErrorCount = 0
    valueErrorCount = 0
    for subdir, dirs, files in os.walk(rootdir):
        if subdir.endswith('train/parsed_pdfs'):
            for x, y, filenames in os.walk(subdir):
                for filename in filenames:
                    try:
                        df1 = pd.DataFrame(read_json(subdir + '/' + filename))
                        if "2019" in subdir or "2020" in subdir or "data_to_drive/2016" in subdir or "data_to_drive/2017" in subdir:
                            df1['accepted'] = True
                        df = df.append(df1[df1.columns & columns_list], ignore_index=True)
                        count += 1
                        print('Added in dataframe: ', count)
                    except InvalidIndexError:
                        invalidIndexErrorCount += 1
                        print('Invalid Index Error count: ', invalidIndexErrorCount)
                        continue
                    except IndexError:
                        indexErrorCount += 1
                        print('Index Error count: ', indexErrorCount)
                        continue
                    except ValueError:
                        valueErrorCount += 1
                        print('Value Error count: ', valueErrorCount)
                        continue

    return df

In [ ]:
def read_reviews_from_json(path_to_json):
    try:
        df = pd.read_json(open(path_to_json))
        # df["id"] = int(df["int"])
    except:
        # df = pd.read_json(open(path_to_json), orient='table')
        with open(path_to_json) as json_data:
            data = json.load(json_data)
            # if "." in str(data["id"]):
                # string_id = str(data["id"]).split('.')[1]
                # data["id"] = int(string_id)
                # data["id"] = int(data["id"])
            if "accepted" in data:
                df = pd.DataFrame({"id": [data['id']], "title": [data['title']], "abstract": [data['abstract']], "reviews": [data['reviews']], "accepted": [data['accepted']]})
            else:
                df = pd.DataFrame({"id": [data['id']], "title": [data['title']], "abstract": [data['abstract']], "reviews": [data['reviews']]})
                # df["id"] = int(data["id"])
    return df

def create_reviews_dataframe():
    rootdir = './drive/MyDrive/data_to_drive/'
    columns_list = ["id", "title", "abstract", "reviews", "accepted"]
    df = pd.DataFrame(columns=columns_list)
    count = 0
    invalidIndexErrorCount = 0
    indexErrorCount = 0
    valueErrorCount = 0
    for subdir, dirs, files in os.walk(rootdir):
        if subdir.endswith('train/reviews'):
            for x, y, filenames in os.walk(subdir):
                for filename in filenames:
                    try:
                        df1 = read_reviews_from_json(subdir + '/' + filename)
                        df = df.append(df1[df1.columns & columns_list], ignore_index=True)
                        # df = df.append(df1)
                        count += 1
                        print('Added in reviews dataframe: ', count)
                    except InvalidIndexError:
                        valueErrorCount += 1
                        print('Invalid Index Error count: ', invalidIndexErrorCount)
                        continue
                    except IndexError:
                        indexErrorCount += 1
                        print('Index Error count: ', indexErrorCount)
                        continue
                    except ValueError:
                        valueErrorCount += 1
                        print('Value Error count: ', valueErrorCount)
                        continue
    df = df.rename({'abstract': 'abstractText'}, axis=1)
    preprocess_reviews_dataframe(df)
    return df

In [ ]:
def realdata(papertexts):
    s = ""
    for i in range(len(papertexts)):
        w_list = papertexts[i].split()
        indexvalue= w_list.index("abstract")+1 if "abstract" in w_list else 0
        s = s+ " ".join( w_list[indexvalue: ] )
    return s

def each_column_word_cloud(df):
    completestring = realdata(df['abstractText'])
    x, y = np.ogrid[:750, :750]
    mask = (x - 375) ** 2 + (y - 375) ** 2 > 390 ** 2
    mask = 255 * mask.astype(int)
    wordcloud = WordCloud(background_color="rgba(255, 255, 255, 0)",
                          mode="RGBA",
                          width=800,
                          height=400,
                          max_words=100,
                          mask=mask
                          ).generate(completestring)
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud)
    plt.show()
    plt.axis('off')

In [ ]:
def SVM_classifier(df):
    y = df.pop('accepted')
    df.pop('name')
    df.pop('creator')
    df.pop('authors')
    df.pop('emails')
    X = df
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)
    svc = svm.SVC(kernel='linear', C=1, gamma='auto')
    svc.fit(xtrain, ytrain)
    score = svc.score(xtrain, ytrain)
    print(score)

def tfidf_on_dataframe(df):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))

def build_model():
    input_comment = Input(shape=(1,), dtype="string", name='input_comment')
    embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_comment)
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    pred = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[input_comment], outputs=pred)
    model.compile(keras.optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy', f1_m])
    # model.run_eagerly = True
    return model

def run_model_elmo(df, model_elmo):
    y = df.pop('accepted')
    X = df
    xtrain, xtest, ytrain, ytest = train_test_split(X.index, y, test_size=0.2)

    # with tf.compat.v1.Session() as session:
    #     tf.compat.v1.keras.backend.set_session(session)
    #     session.run(tf.compat.v1.global_variables_initializer())
    #     session.run(tf.compat.v1.tables_initializer())
    #
    #     history = model_elmo.fit(xtrain, ytrain,validation_data=(xtest, ytest), epochs=10, batch_size=16)
    #     model_elmo.save_weights('./model_elmo_weights.h5')
    #     scores={}
    #     score_gl_test = model_elmo.evaluate([xtest], ytest, verbose=0)
    #     print('Test acc:', score_gl_test[1])
    #     print('Test fscore:', score_gl_test[2])

    with tf.compat.v1.Session() as session:
        tf.compat.v1.keras.backend.set_session(session)
        session.run(tf.compat.v1.global_variables_initializer())
        session.run(tf.compat.v1.tables_initializer())

        train_model(model_elmo, xtrain, xtest, ytrain, ytest)
        # history = model_elmo.fit(xtrain, ytrain,validation_data=(xtest, ytest), epochs=10, batch_size=16)
        model_elmo.save_weights('./model_elmo_weights.h5')
        scores={}
        score_gl_test = model_elmo.evaluate([xtest], ytest, verbose=0)
        print('Test acc:', score_gl_test[1])
        print('Test fscore:', score_gl_test[2])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = create_dataframe_from_parsed_pdfs()
df = pd.read_pickle('/content/drive/MyDrive/parsed_pdf.h5')

In [ ]:
df.head()

,name,abstractText,authors,creator,emails,referenceMentions,references,title,year,Introduction,Related Work,Experiments,Results,Conclusion,Discussion,accepted
0,29539ed932d32f1c56324cded92c07c2-Paper.pdf,"In self-supervised learning, a system is taske...",[Massimiliano Patacchiola],None,"[mpatacch@ed.ac.uk, a.storkey@ed.ac.uk]",64,63,Self-Supervised Relational Reasoning for Repre...,0,Learning useful representations from unlabeled...,NaN,Evaluating self-supervised methods is problema...,NaN,NaN,NaN,True
1,33dd6dba1d56e826aac1cbf23cdcca87-Paper.pdf,"Graph neural networks (GNNs), which learn the ...","[Kaixiong Zhou, Xiao Huang, Yuening Li]",LaTeX with hyperref,"[zkxiong@tamu.edu, xiaohuang@comp.polyu.edu.hk...",64,36,Towards Deeper Graph Neural Networks with Diff...,2020,"Graph neural networks (GNNs) [1, 2, 3] have em...",NaN,We now empirically evaluate the effectiveness ...,NaN,"In this paper, we propose two over-smoothing m...",NaN,True
2,f05da679342107f92111ad9d65959cd3-Paper.pdf,Despite the recent success of Bayesian optimiz...,"[Noémie Jaquier, Leonel Rozo]",LaTeX with hyperref package,"[noemie.jaquier@kit.edu, leonel.rozo@de.bosch....",80,60,High-Dimensional Bayesian Optimization via Nes...,2020,Bayesian optimization (BO) is considered as a ...,NaN,"In this section, we evaluate the proposed HD-G...",NaN,"In this paper, we proposed HD-GaBO, a high-dim...",NaN,True
3,d1dc3a8270a6f9394f88847d7f0050cf-Paper.pdf,Exploration is a key problem in reinforcement ...,"[Jack Parker-Holder, Aldo Pacchiano, Krzysztof...",LaTeX with hyperref package,"[jackph@robots.ox.ac.uk, pacchiano@berkeley.ed...",106,62,Effective Diversity in Population Based Reinfo...,2021,Reinforcement Learning (RL) considers the prob...,"Neuroevolution methods [56], seek to maximize ...",Here evaluate DvD-ES and DvD-TD3 in a variety ...,NaN,NaN,NaN,True
4,fcbc95ccdd551da181207c0c1400c655-Paper.pdf,One paradigm for learning from few labeled exa...,"[Ting Chen, Simon Kornblith, Kevin Swersky, Mo...",PDFium,[iamtingchen@google.com],97,67,Big Self-Supervised Models are Strong Semi-Sup...,2020,Learning from just a few labeled examples whil...,NaN,NaN,NaN,NaN,"In this work, we present a simple framework fo...",True


In [ ]:
#  reviews_df = create_reviews_dataframe()
reviews_df = pd.read_pickle('/content/drive/MyDrive/reviews_pdf.h5')

In [ ]:
 merged_df = df.merge(reviews_df, left_on=['title', 'abstractText', 'accepted'], right_on=['title', 'abstractText', 'accepted'], how='outer')

In [ ]:
merged_df.describe()

,name,abstractText,authors,creator,emails,referenceMentions,references,title,year,Introduction,Related Work,Experiments,Results,Conclusion,Discussion,accepted,id,reviews
count,16359,27302,16359,12749,16359,16359,16359,26125,16359,11878,3888,4771,1502,5283,1777,15503,11098,11098
unique,15210,27027,14476,324,12418,235,157,25599,24,11804,3733,3922,1281,5247,1729,2,11063,506
top,7634ea65a4e6d9041cfd3f7de18e334a-Paper.pdf,present transition based parser jointly produc...,[],LaTeX with hyperref package,[],0,0,RECURRENT NEURAL NETWORKS,2017,,,,,,,False,66,[]
freq,4,4,765,8345,2953,1531,782,7,4254,73,150,827,216,4,45,8188,4,10593


In [ ]:
# df = merged_df[merged_df['accepted'].notna()]
# df.describe()
final_df = merged_df[merged_df['accepted'].notna()]
final_df = final_df[final_df['abstractText'].notna()]
final_df = final_df[final_df['title'].notna()]
del final_df['name']
del final_df['authors']
del final_df['creator']
del final_df['emails']
del final_df['referenceMentions']
del final_df['references']
final_df.describe()

,abstractText,title,year,Introduction,Related Work,Experiments,Results,Conclusion,Discussion,accepted,id,reviews
count,15487,15487,4545,4388,1434,2093,371,2064,659,15487,10942,10942
unique,15433,15365,7,4359,1385,1798,304,2064,650,2,10942,350
top,neural network function approximators achieved...,lie access neural turing machine,2020,,,,,Transferring languages from one style to anoth...,,False,1610.02583,[]
freq,2,3,1472,30,50,295,67,1,10,8188,1,10593


In [ ]:
final_df["accepted"] = final_df["accepted"].astype(int)
final_df['accepted'].value_counts()

0    8188
1    7299
Name: accepted, dtype: int64

In [ ]:

df_new = final_df[final_df['accepted'] == 0]
df_new.head()

print(df_new['abstractText'].iloc[2])
print(df_new['title'].iloc[2])

despite recent advance remaining bottleneck deep generative model necessity extensive training difficulty generalization small number training example problem may addressed conditional generative model trained adapt generative distribution additional input data far idea explored certain limitation restricting input data single object multiple object representing concept work develop new class deep generative model called generative matching network inspired recently proposed matching network one shot learning discriminative task idea meta learning conditioning additional input dataset generative matching network may instantly learn new concept available training conform similar generative process without explicit limitation number additional input object number concept represent experiment omniglot dataset demonstrate generative matching network significantly improve predictive performance fly additional data available model also adapt latent space beneficial context feature extraction

In [ ]:
# df.isna().sum()
final_df.isnull().sum(axis = 0)

abstractText         0
title                0
year             10942
 Introduction    11099
 Related Work    14053
 Experiments     13394
 Results         15116
 Conclusion      13423
 Discussion      14828
accepted             0
id                4545
reviews           4545
dtype: int64

In [ ]:
final_df.notna().sum(axis = 0)

abstractText     15487
title            15487
year              4545
 Introduction     4388
 Related Work     1434
 Experiments      2093
 Results           371
 Conclusion       2064
 Discussion        659
accepted         15487
id               10942
reviews          10942
dtype: int64

In [ ]:
final_df.head()

,abstractText,title,year,Introduction,Related Work,Experiments,Results,Conclusion,Discussion,accepted,id,reviews
0,"In self-supervised learning, a system is taske...",Self-Supervised Relational Reasoning for Repre...,0,Learning useful representations from unlabeled...,NaN,Evaluating self-supervised methods is problema...,NaN,NaN,NaN,1,NaN,NaN
1,"Graph neural networks (GNNs), which learn the ...",Towards Deeper Graph Neural Networks with Diff...,2020,"Graph neural networks (GNNs) [1, 2, 3] have em...",NaN,We now empirically evaluate the effectiveness ...,NaN,"In this paper, we propose two over-smoothing m...",NaN,1,NaN,NaN
2,Despite the recent success of Bayesian optimiz...,High-Dimensional Bayesian Optimization via Nes...,2020,Bayesian optimization (BO) is considered as a ...,NaN,"In this section, we evaluate the proposed HD-G...",NaN,"In this paper, we proposed HD-GaBO, a high-dim...",NaN,1,NaN,NaN
3,Exploration is a key problem in reinforcement ...,Effective Diversity in Population Based Reinfo...,2021,Reinforcement Learning (RL) considers the prob...,"Neuroevolution methods [56], seek to maximize ...",Here evaluate DvD-ES and DvD-TD3 in a variety ...,NaN,NaN,NaN,1,NaN,NaN
4,One paradigm for learning from few labeled exa...,Big Self-Supervised Models are Strong Semi-Sup...,2020,Learning from just a few labeled examples whil...,NaN,NaN,NaN,NaN,"In this work, we present a simple framework fo...",1,NaN,NaN


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000

final_df_copy=final_df.copy()
final_df_copy.reset_index(inplace=True)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(final_df_copy)):
    print(i)
    abstract = re.sub('[^a-zA-Z]', ' ', final_df_copy['abstractText'][i])
    abstract = abstract.lower()
    abstract = abstract.split()
    
    abstract = [ps.stem(word) for word in abstract if not word in stopwords.words('english')]
    abstract = ' '.join(abstract)
    corpus.append(abstract)

Streaming output truncated to the last 5000 lines.
10487
10488
10489
10490
10491
10492
10493
10494
10495
10496
10497
10498
10499
10500
10501
10502
10503
10504
10505
10506
10507
10508
10509
10510
10511
10512
10513
10514
10515
10516
10517
10518
10519
10520
10521
10522
10523
10524
10525
10526
10527
10528
10529
10530
10531
10532
10533
10534
10535
10536
10537
10538
10539
10540
10541
10542
10543
10544
10545
10546
10547
10548
10549
10550
10551
10552
10553
10554
10555
10556
10557
10558
10559
10560
10561
10562
10563
10564
10565
10566
10567
10568
10569
10570
10571
10572
10573
10574
10575
10576
10577
10578
10579
10580
10581
10582
10583
10584
10585
10586
10587
10588
10589
10590
10591
10592
10593
10594
10595
10596
10597
10598
10599
10600
10601
10602
10603
10604
10605
10606
10607
10608
10609
10610
10611
10612
10613
10614
10615
10616
10617
10618
10619
10620
10621
10622
10623
10624
10625
10626
10627
10628
10629
10630
10631
10632
10633
10634
10635
10636
10637
10638
10639
10640
10641
10642
10643
10644
1

In [ ]:
corpus[:5]

['self supervis learn system task achiev surrog object defin altern target set unlabel data aim build use represent use downstream task without costli manual annot work propos novel self supervis formul relat reason allow learner bootstrap signal inform implicit unlabel data train relat head discrimin entiti relat intra reason entiti inter reason result rich descript represent underli neural network backbon use downstream task classif imag retriev evalu propos method follow rigor experiment procedur use standard dataset protocol backbon self supervis relat reason outperform best competitor condit averag accuraci recent state art model link effect method maxim bernoulli log likelihood consid proxi maxim mutual inform result effici object respect commonli use contrast loss',
 'graph neural network gnn learn represent node aggreg neighbor becom effect comput tool downstream applic smooth one key issu limit perform gnn number layer increas stack aggreg would make node represent converg ind

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1300,
  143,
  2997,
  2366,
  857,
  3586,
  4991,
  1119,
  729,
  1690,
  250,
  3734,
  294,
  4196,
  106,
  4284,
  1137,
  2700,
  1137,
  496,
  857,
  136,
  500,
  976,
  3387,
  1870,
  997,
  4353,
  1300,
  143,
  1766,
  738,
  141,
  2063,
  1831,
  4651,
  945,
  4491,
  656,
  294,
  4196,
  4611,
  738,
  1015,
  2353,
  4843,
  738,
  2367,
  141,
  4843,
  4888,
  141,
  2397,
  1013,
  1367,
  2700,
  417,
  1595,
  1872,
  4670,
  1137,
  496,
  857,
  762,
  1515,
  4302,
  4178,
  997,
  3010,
  3819,
  3099,
  2461,
  817,
  1137,
  1131,
  2821,
  2554,
  4670,
  1300,
  143,
  738,
  141,
  1181,
  3765,
  4341,
  2196,
  3905,
  650,
  4427,
  1828,
  2141,
  4203,
  1459,
  436,
  3010,
  4222,
  98,
  113,
  3500,
  698,
  1461,
  4222,
  1160,
  4491,
  2397,
  745,
  1119,
  2935,
  4753,
  1137,
  1419,
  686],
 [4286,
  1595,
  1872,
  1888,
  2997,
  2700,
  2705,
  4984,
  1006,
  3821,
  436,
  1632,
  2478,
  496,
  3863,
  489,
  1931,
  1662,
 

Embedding Representation

In [ ]:
sent_length=150
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1137 1419  686]
 [   0    0    0 ... 1478 2148 1888]
 [   0    0    0 ... 1001 3657 3010]
 ...
 [   0    0    0 ... 2336 3027  754]
 [   0    0    0 ... 2548  732 2366]
 [   0    0    0 ...  173 2393 2355]]


Creating LSTM Model

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
# model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 40)           200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(final_df['accepted'])

X_final.shape,y_final.shape

((15487, 150), (15487,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
163/163 [==============================] - 33s 189ms/step - loss: 0.5887 - accuracy: 0.6893 - val_loss: 0.5031 - val_accuracy: 0.7576
Epoch 2/10
163/163 [==============================] - 30s 186ms/step - loss: 0.4219 - accuracy: 0.8081 - val_loss: 0.4876 - val_accuracy: 0.7678
Epoch 3/10
163/163 [==============================] - 30s 187ms/step - loss: 0.3653 - accuracy: 0.8442 - val_loss: 0.5073 - val_accuracy: 0.7633
Epoch 4/10
163/163 [==============================] - 31s 188ms/step - loss: 0.3318 - accuracy: 0.8622 - val_loss: 0.5588 - val_accuracy: 0.7711
Epoch 5/10
163/163 [==============================] - 31s 188ms/step - loss: 0.2848 - accuracy: 0.8838 - val_loss: 0.5628 - val_accuracy: 0.7636
Epoch 6/10
163/163 [==============================] - 31s 188ms/step - loss: 0.2449 - accuracy: 0.9046 - val_loss: 0.6007 - val_accuracy: 0.7625
Epoch 7/10
163/163 [==============================] - 30s 187ms/step - loss: 0.2054 - accuracy: 0.9241 - val_loss: 0.6324 - val_ac

Performance Metrics and Accuracy for LSTM

In [ ]:
y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1955,  736],
       [ 602, 1818]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7382117002543533

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.73      0.75      2691
           1       0.71      0.75      0.73      2420

    accuracy                           0.74      5111
   macro avg       0.74      0.74      0.74      5111
weighted avg       0.74      0.74      0.74      5111



Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
embedding_vector_features=40
birectional_lstm_model=Sequential()
birectional_lstm_model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
birectional_lstm_model.add(Bidirectional(LSTM(100)))
# birectional_lstm_model.add(Dropout(0.3))
birectional_lstm_model.add(Dense(1,activation='sigmoid'))
birectional_lstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(birectional_lstm_model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 150, 40)           200000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
birectional_lstm_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
163/163 [==============================] - 52s 301ms/step - loss: 0.5455 - accuracy: 0.7221 - val_loss: 0.4820 - val_accuracy: 0.7748
Epoch 2/10
163/163 [==============================] - 48s 297ms/step - loss: 0.4033 - accuracy: 0.8222 - val_loss: 0.4679 - val_accuracy: 0.7801
Epoch 3/10
163/163 [==============================] - 48s 295ms/step - loss: 0.3540 - accuracy: 0.8522 - val_loss: 0.4960 - val_accuracy: 0.7683
Epoch 4/10
163/163 [==============================] - 48s 296ms/step - loss: 0.3246 - accuracy: 0.8672 - val_loss: 0.5153 - val_accuracy: 0.7654
Epoch 5/10
163/163 [==============================] - 48s 295ms/step - loss: 0.2899 - accuracy: 0.8823 - val_loss: 0.5685 - val_accuracy: 0.7638
Epoch 6/10
163/163 [==============================] - 48s 295ms/step - loss: 0.2398 - accuracy: 0.9034 - val_loss: 0.6323 - val_accuracy: 0.7621
Epoch 7/10
163/163 [==============================] - 48s 297ms/step - loss: 0.2088 - accuracy: 0.9176 - val_loss: 0.6158 - val_ac

Performance Metrics and Accuracy for Bidirectional LSTM

In [ ]:
y_pred1=birectional_lstm_model.predict_classes(X_test)

print(confusion_matrix(y_test,y_pred1))

print('Accuracy', accuracy_score(y_test,y_pred1))

print(classification_report(y_test,y_pred1))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1957  734]
 [ 562 1858]]
0.7464292702015262
              precision    recall  f1-score   support

           0       0.78      0.73      0.75      2691
           1       0.72      0.77      0.74      2420

    accuracy                           0.75      5111
   macro avg       0.75      0.75      0.75      5111
weighted avg       0.75      0.75      0.75      5111

